In [1]:
import flowshape as fs
import igl
import numpy as np
import meshplot as mp
import os
import pandas as pd
from src.utilities.fin_shape_utils import plot_mesh
from src.utilities.fin_class_def import FinData
from src.utilities.functions import path_leaf
import glob2 as glob
import meshplot as mp
import trimesh

## First pass did not yield great results
One possible reason is that the SH decomposition was not encoding any size info. I want to see if it is possible to do this by altering the scalar cuvature field by a size factor

### Load fin data

In [2]:
# get list of refined fin mesh objects
root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/"
fin_mesh_list = sorted(glob.glob(os.path.join(root, "point_cloud_data", "processed_fin_data", "*smoothed_fin_mesh*")))

# load metadata
metadata_df = pd.read_csv(os.path.join(root, "metadata", "master_metadata.csv"))
metadata_df["experiment_date"] = metadata_df["experiment_date"].astype(str)
metadata_df.head()

# make savedir 
save_dir = os.path.join(root, "point_cloud_data", "SH_analysis_v2", "")
y_subdir = os.path.join(save_dir, "Y_matrices")
if not os.path.isdir(y_subdir):
    os.makedirs(y_subdir)

### This is the norm utility in flowshape, but returns info I need for recovering original fin

In [3]:
from numpy.linalg import norm

def norm_verbose(verts):
    centroid = np.mean(verts, axis=0)
    verts -= centroid
    radii = norm(verts, axis=1)

    m = np.amax(radii)

    verts /= m
    return verts, centroid, m

In [4]:
# make col names to keep track of SH coefficients

def get_sh_colnames(max_degree):

    col_names = []
    
    for l in range(max_degree):
        
        i1 = l**2
        i2 = (l + 1) ** 2
    
        for m in range(i1, i2):
            col_name = f"sh_l{l:03}_m{m-i1:03}"
            col_names.append(col_name)
            
    return col_names


### Iterate through fin meshes

In [20]:
from tqdm import tqdm
import scipy as sp

max_degree = 30

df_list = []
for file_ind, file_path in enumerate(tqdm(fin_mesh_list)):
    # get colnames
    sh_colnames = get_sh_colnames(max_degree)
    
    # extract relevant metadata
    fname = os.path.basename(file_path)
    well_ind = fname.find("well")
    date_string = fname[:well_ind-1]
    well_num = int(fname[well_ind+4:well_ind+8])
    time_ind = fname.find("time")
    time_num = int(fname[time_ind+4:time_ind+8])
    
    # match this to a row in the metadata df
    date_ft = metadata_df["experiment_date"] == date_string
    well_ft = metadata_df["well_index"] == well_num
    time_ft = metadata_df["time_index"] == time_num
    
    meta_temp = metadata_df.loc[date_ft & well_ft & time_ft, :].reset_index(drop=True)

    cv = meta_temp.loc[0, "chem_i"]
    if isinstance(cv, str):
        cvs = cv.split("_")
        chem_id = cvs[0]
        chem_time = int(cvs[1])
    else:
        chem_id = "WT"
        chem_time = np.nan
    meta_temp.loc[0, "chem_id"] = chem_id
    meta_temp.loc[0, "chem_time"] = chem_time
    
    # make temp DF to store results

    try:
        # load mesh
        fin_mesh = trimesh.load(file_path)
        f, v = fin_mesh.faces.copy(), fin_mesh.vertices.copy()
        
        # normalize
        vn, mu, m = norm_verbose(v.copy())
    
        # map to sphere
        v_sphere = fs.sphere_map(vn, f)
    
        # calculate curvature
        rho = fs.curvature_function(vn, v_sphere, f)
        # rho_norm = fs.curvature_function(vn, v_sphere, f)
    
        # adjust to reflect overall scale of the shape
        rho_scaled = rho / m
        
        v_bary = igl.barycenter(v_sphere, f)
        v_bary = fs.project_sphere(v_bary)
        W = 0.5 * igl.doublearea(v_sphere, f)
        W = sp.sparse.diags(W)
        weights, Y_mat = fs.IRF_scalar(rho_scaled, v_bary, W, max_degree=max_degree)
    
        # perform SH decomposition
        # weights, Y_mat, vs = fs.do_mapping(vn, f, l_max=max_degree)
        
        # store results
        meta_temp.loc[0, "scale"] = m.copy()
        meta_temp.loc[0, ["xc", "yc", "zc"]] = mu.copy()
        meta_temp.loc[0, sh_colnames] = weights.copy()
    
        df_list.append(meta_temp)
    
        # save Y
        yname = fname
        yname.replace("_smoothed_fin_mesh.obj", "y_mat.npy")
        np.save(os.path.join(y_subdir, yname), Y_mat)
        
    except:
        pass

  2%|▏         | 4/224 [00:09<09:08,  2.49s/it]

  7%|▋         | 15/224 [00:36<08:29,  2.44s/it]

 44%|████▍     | 98/224 [04:05<07:15,  3.46s/it]

 45%|████▍     | 100/224 [04:14<07:48,  3.78s/it]

 48%|████▊     | 107/224 [04:37<06:37,  3.40s/it]

...

 92%|█████████▏| 206/224 [10:02<01:05,  3.64s/it]

 95%|█████████▌| 213/224 [10:28<00:42,  3.91s/it]

 96%|█████████▌| 214/224 [10:32<00:39,  3.94s/it]

 96%|█████████▌| 215/224 [10:39<00:43,  4.80s/it]

100%|██████████| 224/224 [11:11<00:00,  3.00s/it]


In [21]:
SH_df = pd.concat(df_list, axis=0, ignore_index=True)
SH_df.head()
SH_df.to_csv(os.path.join(save_dir, "fin_sh_df.csv"), index=False)


### Align shit

In [23]:
# extract array of SH coefficients
max_degree = 30
sh_cols = get_sh_colnames(max_degree=max_degree)
sh_array = SH_df.loc[:, sh_cols].to_numpy()

# extract reference
well_ref = 46
time_ref = 0
date_ref = "20240711_01"

# get location of the reference
date_ft = SH_df["experiment_date"] == date_ref
well_ft = SH_df["well_index"] == well_ref
time_ft = SH_df["time_index"] == time_ref

ref_row = SH_df.loc[date_ft & well_ft & time_ft, :].reset_index(drop=True)
ref_sh_weights = ref_row.loc[0, sh_cols].to_numpy().astype('complex128')

sh_array_aligned = np.zeros_like(sh_array)
# iterate through and align
for i in tqdm(range(sh_array.shape[0])):
    # get ceoffs
    sh2 = sh_array[i, :].astype('complex128')
    # compute alignment
    rot2 = fs.compute_max_correlation(ref_sh_weights, sh2, l_max=max_degree)
    # apply alignment to weights
    sh_array_aligned[i, :] = fs.rotate_weights(rot2, sh2)

  0%|          | 0/223 [00:00<?, ?it/s]/home/nick/miniconda3/envs/flowshape_env/lib/python3.10/site-packages/flowshape/so3.py:161: ComplexWarning: Casting complex values to real discards the imaginary part
  weights_rot[l**2 : (l + 1) ** 2] = r_mat[l].dot(
 18%|█▊        | 40/223 [00:01<00:04, 42.21it/s]/home/nick/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 223/223 [00:04<00:00, 44.95it/s]


In [25]:
SH_df_a = SH_df.copy()
SH_df_a.loc[:, sh_cols] = sh_array_aligned
SH_df_a.to_csv(os.path.join(save_dir, "fin_sh_df_aligned.csv"), index=False)

In [26]:
save_dir

"/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/point_cloud_data/SH_analysis_v2/"

In [30]:
file_ind = 5
file_path = fin_mesh_list[file_ind]

# get colnames
sh_colnames = get_sh_colnames(max_degree)

# load mesh
fin_mesh = trimesh.load(file_path)
f, v = fin_mesh.faces.copy(), fin_mesh.vertices.copy()

# normalize
vn, mu, m = norm_verbose(v.copy())

# map to sphere
v_sphere = fs.sphere_map(vn, f)

# calculate curvature
rho = fs.curvature_function(vn, v_sphere, f)
# rho_norm = fs.curvature_function(vn, v_sphere, f)

# adjust to reflect overall scale of the shape
rho_scaled = rho / m * 50

v_bary = igl.barycenter(v_sphere, f)
v_bary = fs.project_sphere(v_bary)
W = 0.5 * igl.doublearea(v_sphere, f)
W = sp.sparse.diags(W)

weights0, Y_mat = fs.IRF_scalar(rho_scaled, v_bary, W, max_degree=max_degree)
weights1, Y_mat = fs.IRF_scalar(rho, v_bary, W, max_degree=max_degree)

In [31]:
print(weights0[:10])
print(weights1[:10])

[ 2.13622498  0.02124962  0.07594588  0.09806546 -0.15153386 -0.47450078
 -1.02799224 -0.62158335 -0.20100582 -0.09934326]
[ 3.18122274  0.0316445   0.11309705  0.14603709 -0.22566114 -0.7066169
 -1.53086511 -0.92564927 -0.29933378 -0.14793996]


In [33]:
np.divide(weights0[:10], weights1[:10])

array([0.67151066, 0.67151066, 0.67151066, 0.67151066, 0.67151066,
       0.67151066, 0.67151066, 0.67151066, 0.67151066, 0.67151066])